In [47]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, QuantileTransformer, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, balanced_accuracy_score, confusion_matrix

model_df = pd.read_csv('../Data/model_data.csv', index_col = 0)
model_df = model_df.drop('industry', axis = 1)
feature_names = model_df.columns
model_df['office'] = model_df['office'].astype('category')
model_dm = xgb.DMatrix(model_df, enable_categorical = True, feature_names = feature_names)
model_df.head(3)

,current_ratio,operating_cash_flow,debt_to_equity,interest_coverage,operating_margin,return_on_assets,return_on_equity,office,has_interest_payments
0001090872:2011:FY,3.031573,0.477953,1.098471,12.453488,0.533485,0.111737,0.234476,Industrial Applications and Services,1
0001090872:2012:FY,2.445325,-0.621236,1.032015,11.079208,0.525518,0.109434,0.222372,Industrial Applications and Services,1
0001090872:2013:FY,3.110487,0.202247,1.020420,8.887850,0.521233,0.067752,0.136888,Industrial Applications and Services,1


In [23]:
# xgboost_model = xgb.XGBClassifier()
# xgboost_model.train(dtrain = model_dm)
xgboost_model = xgb.train(params = dtrain = model_dm)

TypeError: train() missing 1 required positional argument: 'params'

In [11]:
target_encoding = LabelEncoder()
scaler = QuantileTransformer(n_quantiles = 20000, subsample= 25000)

X = model_df.drop('office', axis = 1).copy()

X = scaler.fit_transform(X)

y = model_df['office'].copy()

y = target_encoding.fit_transform(y)

office_dict = {}

for label in range(len(target_encoding.classes_)):
    office = target_encoding.classes_[label]
    office_dict[label] = office
    
office_dict

{0: 'Industrial Applications and Services',
 1: 'Office of Crypto Assets',
 2: 'Office of Energy & Transportation',
 3: 'Office of Finance',
 4: 'Office of Finance or Office of Crypto Assets',
 5: 'Office of Life Sciences',
 6: 'Office of Manufacturing',
 7: 'Office of Real Estate & Construction',
 8: 'Office of Technology',
 9: 'Office of Trade & Services'}

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [56]:
model = xgb.XGBClassifier(missing = None)

model.fit(X_train, y_train, eval_metric = 'aucpr', verbose = True, early_stopping_rounds = 15, eval_set = [(X_test, y_test)])

# y_pred = model.predict(X_test)

# print(accuracy_score(y_test, y_pred))
# print(balanced_accuracy_score(y_test, y_pred, adjusted = False))

[0]	validation_0-aucpr:0.13428
[1]	validation_0-aucpr:0.14274
[2]	validation_0-aucpr:0.14467
[3]	validation_0-aucpr:0.14687


y:\Anaconda\envs\LHLenvironment\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
y:\Anaconda\envs\LHLenvironment\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[4]	validation_0-aucpr:0.15409
[5]	validation_0-aucpr:0.15556
[6]	validation_0-aucpr:0.15713
[7]	validation_0-aucpr:0.15885
[8]	validation_0-aucpr:0.15989
[9]	validation_0-aucpr:0.16023
[10]	validation_0-aucpr:0.16281
[11]	validation_0-aucpr:0.16371
[12]	validation_0-aucpr:0.16495
[13]	validation_0-aucpr:0.16565
[14]	validation_0-aucpr:0.16851
[15]	validation_0-aucpr:0.16964
[16]	validation_0-aucpr:0.17020
[17]	validation_0-aucpr:0.16997
[18]	validation_0-aucpr:0.16993
[19]	validation_0-aucpr:0.17057
[20]	validation_0-aucpr:0.17025
[21]	validation_0-aucpr:0.17099
[22]	validation_0-aucpr:0.17118
[23]	validation_0-aucpr:0.17078
[24]	validation_0-aucpr:0.17087
[25]	validation_0-aucpr:0.17078
[26]	validation_0-aucpr:0.17096
[27]	validation_0-aucpr:0.16965
[28]	validation_0-aucpr:0.16882
[29]	validation_0-aucpr:0.16963
[30]	validation_0-aucpr:0.17007
[31]	validation_0-aucpr:0.16997
[32]	validation_0-aucpr:0.16990
[33]	validation_0-aucpr:0.17030
[34]	validation_0-aucpr:0.17095
[35]	validatio

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=None, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [50]:
confusion_matrix(model)

TypeError: confusion_matrix() missing 1 required positional argument: 'y_pred'